In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from WindPy import w
import empyrical
%matplotlib inline

C:\Users\zhangyang01\AppData\Local\Continuum\anaconda2\envs\py36\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
C:\Users\zhangyang01\AppData\Local\Continuum\anaconda2\envs\py36\lib\site-packages\empyrical\utils.py:32: UserWarning: Unable to import pandas_datareader. Suppressing import error and continuing. All data reading functionality will raise errors; but has been deprecated and will be removed in a later version.
  warnings.warn(msg)


In [2]:
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [3]:
start_date, end_date = '2017-01-01', '2020-08-28'

# 基金池收益测算

In [4]:
stocks = pd.read_excel('../data/养老型基金测算.xlsx', sheet_name='股票基金池')

In [5]:
data = w.wss(stocks['代码'].tolist(), "risk_returnyearly,risk_stdevyearly", "startDate=%s;endDate=%s;period=2;returnType=1"%(start_date, end_date))

In [6]:
stocks['ret'], stocks['vol'] = data.Data

In [7]:
stocks_ret, stocks_vol = stocks['ret'].mean(), stocks['vol'].mean()

In [8]:
stocks_ret, stocks_vol

(28.220328645788456, 19.945177388825076)

In [9]:
bonds = pd.read_excel('../data/养老型基金测算.xlsx', sheet_name='债券基金池')

In [10]:
data = w.wss(bonds['代码'].tolist(), "risk_returnyearly,risk_stdevyearly", "startDate=%s;endDate=%s;period=2;returnType=1"%(start_date, end_date))

In [11]:
bonds['ret'], bonds['vol'] = data.Data

In [12]:
bonds_ret, bonds_vol = bonds['ret'].mean(), bonds['vol'].mean()

In [13]:
bonds_ret, bonds_vol

(4.412626587192667, 1.4718549973348858)

# TRF测算

In [14]:
def risk_ret(w1, w2):
    return (w1*stocks_ret+w2*bonds_ret)/100, (w1*stocks_vol+w2*bonds_vol)/100

In [15]:
risk_ret(0.1, 0.9)

(0.06793396793052246, 0.033191872364839045)

In [16]:
risk_ret(0.2, 0.8)

(0.09174166998911826, 0.05166519475632924)

In [17]:
risk_ret(0.3, 0.7)

(0.11554937204771402, 0.07013851714781943)

In [18]:
risk_ret(0.4, 0.6)

(0.13935707410630985, 0.08861183953930964)

# TDF测算

In [22]:
tdf = pd.read_excel('../data/养老型基金测算.xlsx', sheet_name='TDF风险收益测算')

In [23]:
for i in tdf.index:
    weights = tdf.loc[tdf.index[i:], ['权益资产比例', '固收资产比例']]
    pf_ret = weights.mul([stocks_ret, bonds_ret], axis=1).sum(axis=1)
#     pf_vol = weights.mul([stocks_vol, bonds_vol], axis=1).sum(axis=1)
    tdf.loc[i, '年化收益率'] = empyrical.annual_return(pf_ret, period='yearly')
    tdf.loc[i, '年化波动率'] = empyrical.annual_volatility(pf_ret, period='yearly')

In [24]:
tdf.to_excel('../data/results.xlsx', index=False)